## Matrix Factorisation

In [1]:
# pip install scikit-surprise
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [2]:
# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin('ml-100k')

In [3]:
# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)

In [4]:
# We'll use the famous SVD algorithm.
algo = SVD()

In [5]:
# Train the algorithm on the trainset
algo.fit(trainset)

In [7]:
#algo.test?

In [8]:
# Predict ratings for the testset
predictions = algo.test(testset)

In [9]:
type(predictions)

list

In [13]:
predictions[2]

Prediction(uid='624', iid='249', r_ui=3.0, est=3.5416168966595842, details={'was_impossible': False})

In [11]:
# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.9443


0.9443121966011361